In [27]:
from scikit_roughsets.rs_reduction import RoughSetsSelector
import numpy as np
import pandas as pd

In [28]:
from sklearn.preprocessing import LabelEncoder
df = pd.read_csv('data/segment/segmentall.csv')
df.drop('REGION-PIXEL-COUNT', axis=1, inplace=True)
# print(df.isnull().sum())
labelencoder=LabelEncoder()

df['target'] = labelencoder.fit_transform(df['target'])


In [29]:
feature_columns = [ col for col in df.columns if col not in ['id','target']]
# feature_columns=['f4','f5','f6','f7','f17','f18']
for col in feature_columns:
    df[col] = pd.qcut(df[col], q=3, labels=False, duplicates='drop')

In [30]:
df

id  target  REGION-CENTROID-COL  REGION-CENTROID-ROW  \
0        1       0                    1                    1   
1        2       0                    2                    1   
2        3       0                    1                    1   
3        4       0                    0                    1   
4        5       0                    0                    1   
...    ...     ...                  ...                  ...   
2305  2306       1                    0                    2   
2306  2307       1                    0                    2   
2307  2308       1                    1                    2   
2308  2309       1                    1                    2   
2309  2310       1                    1                    2   

      SHORT-LINE-DENSITY-5  SHORT-LINE-DENSITY-2  VEDGE-MEAN  VEDGE-SD  \
0                        0                     0           0         0   
1                        0                     0           0         0   
2                        0                     0           0         0   
3                        0                     0           0         0   
4                        0                     0           0         0   
...                    ...                   ...         ...       ...   
2305                     0                     0           1         1   
2306                     0                     0           1         2   
2307                     0                     0           0         0   
2308                     0                     0           2         2   
2309                     0                     0           1         1   

      HEDGE-MEAN  HEDGE-SD  INTENSITY-MEAN  RAWRED-MEAN  RAWBLUE-MEAN  \
0              0         0               0            0             0   
1              0         0               0            0             0   
2              0         0               0            0             0   
3              1         0               0            0             0   
4              0         0               0            0             0   
...          ...       ...             ...          ...           ...   
2305           0         1               0            0             0   
2306           0         0               0            0             0   
2307           0         0               0            0             0   
2308           1         0               0            0             0   
2309           1         1               0            0             1   

      RAWGREEN-MEAN  EXRED-MEAN  EXBLUE-MEAN  EXGREEN-MEAN  VALUE-MEAN  \
0                 0           2            0             1           0   
1                 0           2            0             1           0   
2                 0           2            0             1           0   
3                 0           2            0             1           0   
4                 0           2            0             1           0   
...             ...         ...          ...           ...         ...   
2305              0           2            1             1           0   
2306              0           2            1             2           0   
2307              0           2            1             1           0   
2308              0           2            1             1           0   
2309              0           2            1             1           0   

      SATURATION-MEAN  HUE-MEAN  
0                   2         2  
1                   2         2  
2                   2         2  
3                   2         2  
4                   2         2  
...               ...       ...  
2305                2         1  
2306                2         1  
2307                2         1  
2308                2         1  
2309                2         1  

[2310 rows x 20 columns]

In [31]:
y_series = df['target']
y = y_series.to_numpy().reshape(-1, 1)
feature_columns = [ col for col in df.columns if col not in ['id','target']]
# for col in feature_columns:
#     df[col] = pd.qcut(df[col], q=5, labels=False, duplicates='drop')
# feature_columns = [ col for col in df.columns if col not in ['id','target','REGION-PIXEL-COUNT']]
# print(df[feature_columns].isnull().sum())

# for col in feature_columns:
#     # 获取该特征的众数
#     mode = df[col].mode()[0]
#     # 用该特征的众数填充空值
#     df[col].fillna(mode, inplace=True)

X = np.array(df[feature_columns])
y=np.array(y)
selector = RoughSetsSelector()
X_selected = selector.fit(X, y).transform(X)
print(X_selected.shape)

(2310, 12)


In [32]:
data=pd.DataFrame(X)
data

0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15  16  17
0      1   1   0   0   0   0   0   0   0   0   0   0   2   0   1   0   2   2
1      2   1   0   0   0   0   0   0   0   0   0   0   2   0   1   0   2   2
2      1   1   0   0   0   0   0   0   0   0   0   0   2   0   1   0   2   2
3      0   1   0   0   0   0   1   0   0   0   0   0   2   0   1   0   2   2
4      0   1   0   0   0   0   0   0   0   0   0   0   2   0   1   0   2   2
...   ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
2305   0   2   0   0   1   1   0   1   0   0   0   0   2   1   1   0   2   1
2306   0   2   0   0   1   2   0   0   0   0   0   0   2   1   2   0   2   1
2307   1   2   0   0   0   0   0   0   0   0   0   0   2   1   1   0   2   1
2308   1   2   0   0   2   2   1   0   0   0   0   0   2   1   1   0   2   1
2309   1   2   0   0   1   1   1   1   0   0   1   0   2   1   1   0   2   1

[2310 rows x 18 columns]

In [33]:
common_columns = []
for i in range(X.shape[1]):
    for j in range(X_selected.shape[1]):
        if (X[:, i] == X_selected[:, j]).all():
            common_columns.append(i)
print(common_columns)

[0, 1, 4, 5, 6, 7, 11, 13, 14, 15, 16, 17]


In [34]:
df = pd.read_csv('data/segment/segmentall.csv')
original_features = []
for col_idx in common_columns:
    original_features.append(feature_columns[col_idx])

In [35]:
original_features

['REGION-CENTROID-COL',
 'REGION-CENTROID-ROW',
 'VEDGE-MEAN',
 'VEDGE-SD',
 'HEDGE-MEAN',
 'HEDGE-SD',
 'RAWGREEN-MEAN',
 'EXBLUE-MEAN',
 'EXGREEN-MEAN',
 'VALUE-MEAN',
 'SATURATION-MEAN',
 'HUE-MEAN']

In [36]:
from sklearn import svm
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold

df = pd.read_csv('data/segment/segmentall.csv')
for col in feature_columns:
    labelencoder=LabelEncoder()
    df[col] = labelencoder.fit_transform(df[col])
feature_columns = [col for col in df.columns if col not in ['id', 'target']]
Y = df["target"]
X = df[original_features]
def SVM(trainX, trainY, testX, testY):
    scaler = StandardScaler()
    scaler.fit(trainX)
    trainX3 = scaler.transform(trainX)
    testX3 = scaler.transform(testX)
    clf = svm.SVC(C=10, gamma=0.8, max_iter=1000)
    clf.fit(trainX3, trainY)
    test_result = clf.predict(testX3)
    precision = np.sum(test_result == testY) / testY.shape[0]
    return precision

        # Use stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = []
for train_index, test_index in skf.split(X, Y):
    trainX, testX = X.iloc[train_index], X.iloc[test_index]
    trainY, testY = Y.iloc[train_index], Y.iloc[test_index]
    score = SVM(trainX, trainY, testX, testY)
    scores.append(score)
avg_score = np.mean(scores)
print(avg_score)

0.9658008658008658


In [37]:
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

df = pd.read_csv('data/segment/segmentall.csv')
feature_columns = [ col for col in df.columns if col not in ['id']]
for col in feature_columns:
    labelencoder=LabelEncoder()
    df[col] = labelencoder.fit_transform(df[col])
feature_columns = [col for col in df.columns if col not in ['id','target']]
X = df[original_features]
Y = df['target']
dt = DecisionTreeClassifier(max_depth=5)
kf = KFold(n_splits=10, shuffle=True, random_state=42)

accuracy_scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

    dt.fit(X_train, y_train)

    y_pred = dt.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print(mean_accuracy)

0.825108225108225


In [38]:
from sklearn.neighbors import KNeighborsClassifier

k = 5
knn = KNeighborsClassifier(n_neighbors=k)
df = pd.read_csv('data/segment/segmentall.csv')
for col in feature_columns:
    labelencoder=LabelEncoder()
    df[col] = labelencoder.fit_transform(df[col])
feature_columns = [col for col in df.columns if col not in ['id','target']]
Y=df["target"]
X=df[original_features]
kf = KFold(n_splits=10, shuffle=True, random_state=42)
accuracy_scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print(mean_accuracy)

0.9160173160173161
